In [ ]:
import mysql.connector
from sqlalchemy import create_engine, Column, Integer, String, Float, MetaData, Table
from sqlalchemy.ext.declarative import declarative_base
import pandas as pd
import json
import os

# MySQL connection details
host = 'localhost'
user = 'root'
password = '1611'
database = 'PhonePePulse_DVandExp'

# Create a connection
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)


def convert_state_name(input_string):
    # Replace hyphens and special characters with spaces, then capitalize each word
    formatted_string = input_string.replace('-', ' ').replace('&', '&')
    words = formatted_string.split()
    capitalized_words = [word.capitalize() for word in words]
    return ' '.join(capitalized_words)

# Create a SQLAlchemy engine
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

# Path to data
path = "C:\\Users\\HP\\GUVI PROJ\\pulse\\data\\aggregated\\transaction\\country\\india\\state\\"
Agg_state_list = os.listdir(path)

# Prepare the DataFrame
clm = {'State': [], 'Year': [], 'Quater': [], 'Transacion_type': [], 'Transacion_count': [], 'Transacion_amount': []}

for i in Agg_state_list:
    p_i = path + i + "/"
    Agg_yr = os.listdir(p_i)
    for j in Agg_yr:
        p_j = p_i + j + "/"
        Agg_yr_list = os.listdir(p_j)
        for k in Agg_yr_list:
            p_k = p_j + k
            with open(p_k, 'r') as Data:
                D = json.load(Data)
                for z in D['data']['transactionData']:
                    Name = z['name']
                    count = z['paymentInstruments'][0]['count']
                    amount = z['paymentInstruments'][0]['amount']
                    clm['Transacion_type'].append(Name)
                    clm['Transacion_count'].append(count)
                    clm['Transacion_amount'].append(amount)
                    clm['State'].append(i)
                    clm['Year'].append(j)
                    clm['Quater'].append(int(k.strip('.json')))

# Create DataFrame
Agg_Trans = pd.DataFrame(clm)
Agg_Trans['State'] = Agg_Trans['State'].apply(convert_state_name)
# Store DataFrame to MySQL table (ensure the table name is in lowercase)
table_name = 'agg_transaction'
Agg_Trans.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

# Close the connection
conn.close()

# Display the table from MySQL
# Reopen the connection using SQLAlchemy for reading the table
conn = engine.connect()

# Read the table into a DataFrame
df = pd.read_sql_table(table_name, con=conn)

# Display the DataFrame
print(df)

# Close the connection
conn.close()


In [ ]:
import os
import json
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# MySQL connection details
host = 'localhost'
user = 'root'
password = '1611'
database = 'PhonePePulse_DVandExp'

# Create a connection to MySQL
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

# Define the path to the directory containing JSON files
path = "C:\\Users\\HP\\GUVI PROJ\\pulse\\data\\aggregated\\user\\country\\india\\state\\"

# Dictionary to store extracted data
clm_user = {'State': [], 'Year': [], 'Quarter': [], 'Brand_Name': [], 'Num_of_Reg_Users': [], 'Percent_Share': []}

# Function to convert state names for geovisualization
def convert_state_name(input_string):
    # Replace hyphens and special characters with spaces, then capitalize each word
    formatted_string = input_string.replace('-', ' ').replace('&', '&')
    words = formatted_string.split()
    capitalized_words = [word.capitalize() for word in words]
    return ' '.join(capitalized_words)

# Iterate through each state directory
for state in os.listdir(path):
    state_path = os.path.join(path, state)
    if os.path.isdir(state_path):
        for year in os.listdir(state_path):
            year_path = os.path.join(state_path, year)
            if os.path.isdir(year_path):
                for quarter_file in os.listdir(year_path):
                    quarter_path = os.path.join(year_path, quarter_file)
                    if quarter_file.endswith('.json'):
                        with open(quarter_path, 'r') as file:
                            data = json.load(file)
                            if data['data'] and 'usersByDevice' in data['data'] and data['data']['usersByDevice'] is not None:
                                for device in data['data']['usersByDevice']:
                                    clm_user['State'].append(state)
                                    clm_user['Year'].append(year)
                                    clm_user['Quarter'].append(int(quarter_file.strip('.json')))
                                    clm_user['Brand_Name'].append(device['brand'])
                                    clm_user['Num_of_Reg_Users'].append(device['count'])
                                    clm_user['Percent_Share'].append(device['percentage'])

# Create DataFrame
Agg_User = pd.DataFrame(clm_user)

# Update the 'State' column to be suitable for geovisualization
Agg_User['State'] = Agg_User['State'].apply(convert_state_name)

# Store the updated DataFrame into the MySQL table
table_name = 'agg_user'
Agg_User.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

# Close the initial MySQL connection
conn.close()

# Reopen the connection using SQLAlchemy for reading the table
conn = engine.connect()

# Read the table into a DataFrame
df = pd.read_sql_table(table_name, con=conn)

# Display the DataFrame
print(df)

# Close the connection
conn.close()


In [ ]:
import os
import json
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# MySQL connection details
host = 'localhost'
user = 'root'
password = '1611'
database = 'PhonePePulse_DVandExp'

# Create a connection to MySQL
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

# Define the path to the directory containing JSON files
path = "C:\\Users\\HP\\GUVI PROJ\\pulse\\data\\map\\transaction\\hover\\country\\india\\state\\"

# Dictionary to store extracted data
clm_map_ag = {'State': [], 'Year': [], 'Quarter': [], 'District': [], 'Num_of_Transactions_in_that_Yr': [], 'Total_Trans_iin_that_year': []}

# Function to convert state names for geovisualization
def convert_state_name(input_string):
    # Replace hyphens and special characters with spaces, then capitalize each word
    formatted_string = input_string.replace('-', ' ').replace('&', '&')
    words = formatted_string.split()
    capitalized_words = [word.capitalize() for word in words]
    return ' '.join(capitalized_words)

# Iterate through each state directory
for state in os.listdir(path):
    state_path = os.path.join(path, state)
    if os.path.isdir(state_path):
        for year in os.listdir(state_path):
            year_path = os.path.join(state_path, year)
            if os.path.isdir(year_path):
                for quarter_file in os.listdir(year_path):
                    quarter_path = os.path.join(year_path, quarter_file)
                    if quarter_file.endswith('.json'):
                        with open(quarter_path, 'r') as file:
                            data = json.load(file)
                            for item in data['data']['hoverDataList']:
                                name = item['name']
                                metric = item['metric'][0]
                                count = metric['count']
                                amount = metric['amount']
                                clm_map_ag['State'].append(state)
                                clm_map_ag['Year'].append(year)
                                clm_map_ag['Quarter'].append(int(quarter_file.strip('.json')))
                                clm_map_ag['District'].append(name)
                                clm_map_ag['Num_of_Transactions_in_that_Yr'].append(count)
                                clm_map_ag['Total_Trans_iin_that_year'].append(amount)
                            

# Create DataFrame
Map_Trans = pd.DataFrame(clm_map_ag)

# Update the 'State' column to be suitable for geovisualization
Map_Trans['State'] = Map_Trans['State'].apply(convert_state_name)
Map_Trans['District'] = Map_Trans['District'].apply(convert_state_name)
# Store the updated DataFrame into the MySQL table
table_name = 'map_trans'
Map_Trans.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

# Close the initial MySQL connection
conn.close()

# Reopen the connection using SQLAlchemy for reading the table
conn = engine.connect()

# Read the table into a DataFrame
df = pd.read_sql_table(table_name, con=conn)

# Display the DataFrame
print(df)

# Close the connection
conn.close()


In [ ]:
import os
import json
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# MySQL connection details
host = 'localhost'
user = 'root'
password = '1611'
database = 'PhonePePulse_DVandExp'

# Create a connection to MySQL
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")


def convert_state_name(input_string):
    # Replace hyphens and special characters with spaces, then capitalize each word
    formatted_string = input_string.replace('-', ' ').replace('&', '&')
    words = formatted_string.split()
    capitalized_words = [word.capitalize() for word in words]
    return ' '.join(capitalized_words)



path="C:\\Users\\HP\\GUVI PROJ\\pulse\\data\\map\\user\\hover\\country\\india\\state\\"
clm_map_user = {'State': [], 'Dist_Name':[], 'Year': [], 'Quarter': [], 'Num_of_reg_users': [], 'Num_of_app_opens': []}
for state in os.listdir(path):
    state_path = os.path.join(path, state)
    if os.path.isdir(state_path):
        for year in os.listdir(state_path):
            year_path = os.path.join(state_path, year)
            if os.path.isdir(year_path):
                for quarter_file in os.listdir(year_path):
                    quarter_path = os.path.join(year_path, quarter_file)
                    if quarter_file.endswith('.json'):
                        with open(quarter_path, 'r') as file:
                            data = json.load(file)
                            
                            hover_data = data['data']['hoverData']
                            for district, metrics in hover_data.items():
                                reg_users = metrics['registeredUsers']
                                app_opens = metrics['appOpens']
                                dt_name=district
                                clm_map_user['State'].append(state)
                                clm_map_user['Year'].append(year)
                                clm_map_user['Dist_Name'].append(dt_name)
                                clm_map_user['Quarter'].append(int(quarter_file.strip('.json')))
                                clm_map_user['Num_of_reg_users'].append(reg_users)
                                clm_map_user['Num_of_app_opens'].append(app_opens)
                            
      

                               
Map_User = pd.DataFrame(clm_map_user)

# Update the 'State' column to be suitable for geovisualization
Map_User['State'] = Map_User['State'].apply(convert_state_name)
Map_User['Dist_Name'] = Map_User['Dist_Name'].apply(convert_state_name)
#Map_User['Num_of_reg_users'] = Map_Trans['Num_of_reg_users'].apply(convert_state_name)                               


table_name = 'map_user'
Map_User.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

# Close the initial MySQL connection
conn.close()

# Reopen the connection using SQLAlchemy for reading the table
conn = engine.connect()

# Read the table into a DataFrame
df = pd.read_sql_table(table_name, con=conn)

# Display the DataFrame
print(df)

# Close the connection
conn.close()



In [ ]:
import os
import json
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# MySQL connection details
host = 'localhost'
user = 'root'
password = '1611'
database = 'PhonePePulse_DVandExp'

# Create a connection to MySQL
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

def convert_state_name(input_string):
    # Replace hyphens and special characters with spaces, then capitalize each word
    formatted_string = input_string.replace('-', ' ').replace('&', '&')
    words = formatted_string.split()
    capitalized_words = [word.capitalize() for word in words]
    return ' '.join(capitalized_words)

path = "C:\\Users\\HP\\GUVI PROJ\\pulse\\data\\top\\transaction\\country\\india\\state\\"
clm_top_trans_state = {'State': [], 'Year': [], 'Quarter': [], 'District': [], 'Tot_No_of_Trans': [], 'Tot_Val_of_Trans': []}

for state in os.listdir(path):
    state_path = os.path.join(path, state)
    if os.path.isdir(state_path):
        for year in os.listdir(state_path):
            year_path = os.path.join(state_path, year)
            if os.path.isdir(year_path):
                for quarter_file in os.listdir(year_path):
                    quarter_path = os.path.join(year_path, quarter_file)
                    if quarter_file.endswith('.json'):
                        with open(quarter_path, 'r') as file:
                            data = json.load(file)
                            
                            # Check if 'data' key exists
                            if 'data' in data:
                                districts_data = data['data'].get('districts', [])
                                
                                for district_data in districts_data:
                                    entity_name = state
                                    district_name = district_data['entityName']
                                    metric = district_data['metric']
                                    count = metric['count']
                                    amount = metric['amount']
                                    clm_top_trans_state['State'].append(entity_name)
                                    clm_top_trans_state['Year'].append(year)
                                    clm_top_trans_state['Quarter'].append(int(quarter_file.strip('.json')))
                                    clm_top_trans_state['District'].append(district_name)
                                    clm_top_trans_state['Tot_No_of_Trans'].append(count)
                                    clm_top_trans_state['Tot_Val_of_Trans'].append(amount)

# Create DataFrame
Top_Trans_State = pd.DataFrame(clm_top_trans_state)

# Update the 'State' column to be suitable for geovisualization
Top_Trans_State['State'] = Top_Trans_State['State'].apply(convert_state_name)
Top_Trans_State['District'] = Top_Trans_State['District'].apply(convert_state_name)

table_name = 'top_trans_state'
Top_Trans_State.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

# Close the initial MySQL connection
conn.close()

# Reopen the connection using SQLAlchemy for reading the table
conn = engine.connect()

# Read the table into a DataFrame
df = pd.read_sql_table(table_name, con=conn)

# Display the DataFrame
print(df)

# Close the connection
conn.close()


In [ ]:
import os
import json
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# MySQL connection details
host = 'localhost'
user = 'root'
password = '1611'
database = 'PhonePePulse_DVandExp'

# Create a connection to MySQL
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

def convert_state_name(input_string):
    # Replace hyphens and special characters with spaces, then capitalize each word
    formatted_string = input_string.replace('-', ' ').replace('&', '&')
    words = formatted_string.split()
    capitalized_words = [word.capitalize() for word in words]
    return ' '.join(capitalized_words)

path = "C:\\Users\\HP\\GUVI PROJ\\pulse\\data\\top\\transaction\\country\\india\\state\\"
clm_top_trans_dist = {'District_Name': [],'Year': [], 'Quarter': [],  'Tot_No_of_DTrans': [], 'Tot_Val_of_DTrans': []}

for state in os.listdir(path):
    state_path = os.path.join(path, state)
    if os.path.isdir(state_path):
        for year in os.listdir(state_path):
            year_path = os.path.join(state_path, year)
            if os.path.isdir(year_path):
                for quarter_file in os.listdir(year_path):
                    quarter_path = os.path.join(year_path, quarter_file)
                    if quarter_file.endswith('.json'):
                        with open(quarter_path, 'r') as file:
                            data = json.load(file)
                            
                            # Check if 'data' key exists
                            if 'data' in data:
                                districts = data['data']['districts']
                                
                                for district in districts:
                                    entity_name = district['entityName']                                 
                                    metric = district['metric']
                                    count = metric['count']
                                    amount = metric['amount']                               
                                    clm_top_trans_dist['Year'].append(year)
                                    clm_top_trans_dist['Quarter'].append(int(quarter_file.strip('.json')))
                                    clm_top_trans_dist['District_Name'].append(district_name)
                                    clm_top_trans_dist['Tot_No_of_DTrans'].append(count)
                                    clm_top_trans_dist['Tot_Val_of_DTrans'].append(amount)

# Create DataFrame
Top_Trans_Dist = pd.DataFrame(clm_top_trans_dist)

# Update the 'State' column to be suitable for geovisualization
Top_Trans_Dist['District_Name'] = Top_Trans_Dist['District_Name'].apply(convert_state_name)

table_name = 'top_trans_dist'
Top_Trans_Dist.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

# Close the initial MySQL connection
conn.close()

# Reopen the connection using SQLAlchemy for reading the table
conn = engine.connect()

# Read the table into a DataFrame
df = pd.read_sql_table(table_name, con=conn)

# Display the DataFrame
print(df)

# Close the connection
conn.close()


In [ ]:
import os
import json
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# MySQL connection details
host = 'localhost'
user = 'root'
password = '1611'
database = 'PhonePePulse_DVandExp'

# Create a connection to MySQL
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

def convert_state_name(input_string):
    # Replace hyphens and special characters with spaces, then capitalize each word
    formatted_string = input_string.replace('-', ' ').replace('&', '&')
    words = formatted_string.split()
    capitalized_words = [word.capitalize() for word in words]
    return ' '.join(capitalized_words)

path = "C:\\Users\\HP\\GUVI PROJ\\pulse\\data\\top\\transaction\\country\\india\\state\\"
clm_top_trans_pin = {'Pincode': [], 'State':[],'Year': [], 'Quarter': [],  'Tot_No_of_PTrans': [], 'Tot_Val_of_PTrans': []}

for state in os.listdir(path):
    state_path = os.path.join(path, state)
    if os.path.isdir(state_path):
        for year in os.listdir(state_path):
            year_path = os.path.join(state_path, year)
            if os.path.isdir(year_path):
                for quarter_file in os.listdir(year_path):
                    quarter_path = os.path.join(year_path, quarter_file)
                    if quarter_file.endswith('.json'):
                        with open(quarter_path, 'r') as file:
                            data = json.load(file)
                            
                            # Check if 'data' key exists
                            if 'data' in data and 'pincodes' in data['data']:
                                pincodes = data['data']['pincodes']

                            for pincode in pincodes:
                                entity_name = pincode['entityName']
                                metric = pincode['metric']
                                count = metric['count']
                                amount = metric['amount']
                                clm_top_trans_pin['Year'].append(year)
                                clm_top_trans_pin['Quarter'].append(int(quarter_file.strip('.json')))
                                clm_top_trans_pin['Pincode'].append(entity_name)
                                clm_top_trans_pin['Tot_No_of_PTrans'].append(count)
                                clm_top_trans_pin['Tot_Val_of_PTrans'].append(amount)
                                clm_top_trans_pin['State'].append(state)

# Create DataFrame
Top_Trans_Pin = pd.DataFrame(clm_top_trans_pin)

# Update the 'State' column to be suitable for geovisualization


table_name = 'top_trans_pin'
Top_Trans_Pin['State']=Top_Trans_Pin['State'].apply(convert_state_name)
Top_Trans_Pin.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

# Close the initial MySQL connection
conn.close()

# Reopen the connection using SQLAlchemy for reading the table
conn = engine.connect()

# Read the table into a DataFrame
df = pd.read_sql_table(table_name, con=conn)

# Display the DataFrame
print(df)

# Close the connection
conn.close()


In [ ]:
import os
import json
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# MySQL connection details
host = 'localhost'
user = 'root'
password = '1611'
database = 'PhonePePulse_DVandExp'

# Create a connection to MySQL
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

def convert_state_name(input_string):
    # Replace hyphens and special characters with spaces, then capitalize each word
    formatted_string = input_string.replace('-', ' ').replace('&', '&')
    words = formatted_string.split()
    capitalized_words = [word.capitalize() for word in words]
    return ' '.join(capitalized_words)





path="C:\\Users\HP\\GUVI PROJ\\pulse\\data\\top\\user\\country\\india\\state"
clm_top_user_dis = {'District_Name': [],'State':[],'Year': [], 'Quarter': [],  'Reg_Users': []}

for state in os.listdir(path):
    state_path = os.path.join(path, state)
    if os.path.isdir(state_path):
        for year in os.listdir(state_path):
            year_path = os.path.join(state_path, year)
            if os.path.isdir(year_path):
                for quarter_file in os.listdir(year_path):
                    quarter_path = os.path.join(year_path, quarter_file)
                    if quarter_file.endswith('.json'):
                        with open(quarter_path, 'r') as file:
                            data = json.load(file)

                            if 'data' in data and 'districts' in data['data']:
                                districts = data['data']['districts']

                            for dis in districts:
                                name = dis['name']
                                reg_users = dis['registeredUsers']                               
                                clm_top_user_dis['Year'].append(year)
                                clm_top_user_dis['Quarter'].append(int(quarter_file.strip('.json')))
                                clm_top_user_dis['District_Name'].append(name)
                                clm_top_user_dis['State'].append(state)
                                clm_top_user_dis['Reg_Users'].append(reg_users)
                                
# Create DataFrame
Top_User_dis = pd.DataFrame(clm_top_user_dis)

# Update the 'State' column to be suitable for geovisualization


table_name = 'top_user_dis'
Top_User_dis['State'] = Top_User_dis['State'].apply(convert_state_name)
Top_User_dis['District_Name'] = Top_User_dis['District_Name'].apply(convert_state_name)
Top_User_dis.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

# Close the initial MySQL connection
conn.close()

# Reopen the connection using SQLAlchemy for reading the table
conn = engine.connect()

# Read the table into a DataFrame
df = pd.read_sql_table(table_name, con=conn)

# Display the DataFrame
print(df)

# Close the connection
conn.close()
                            

In [ ]:
import os
import json
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# MySQL connection details
host = 'localhost'
user = 'root'
password = '1611'
database = 'PhonePePulse_DVandExp'

# Create a connection to MySQL
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

def convert_state_name(input_string):
    # Replace hyphens and special characters with spaces, then capitalize each word
    formatted_string = input_string.replace('-', ' ').replace('&', '&')
    words = formatted_string.split()
    capitalized_words = [word.capitalize() for word in words]
    return ' '.join(capitalized_words)





path="C:\\Users\HP\\GUVI PROJ\\pulse\\data\\top\\user\\country\\india\\state"
clm_top_user_pin = {'Pin_Code': [],'State':[],'Year': [], 'Quarter': [],  'Reg_Users': []}

for state in os.listdir(path):
    state_path = os.path.join(path, state)
    if os.path.isdir(state_path):
        for year in os.listdir(state_path):
            year_path = os.path.join(state_path, year)
            if os.path.isdir(year_path):
                for quarter_file in os.listdir(year_path):
                    quarter_path = os.path.join(year_path, quarter_file)
                    if quarter_file.endswith('.json'):
                        with open(quarter_path, 'r') as file:
                            data = json.load(file)

                            if 'data' in data and 'pincodes' in data['data']:
                                pincodes = data['data']['pincodes']

                            for pin in pincodes:
                                code= pin['name']
                                reg_users = pin['registeredUsers']                               
                                clm_top_user_pin['Year'].append(year)
                                clm_top_user_pin['Quarter'].append(int(quarter_file.strip('.json')))
                                clm_top_user_pin['Pin_Code'].append(code)
                                clm_top_user_pin['Reg_Users'].append(reg_users)
                                clm_top_user_pin['State'].append(state)
                                
# Create DataFrame
Top_User_pin = pd.DataFrame(clm_top_user_pin)

# Update the 'State' column to be suitable for geovisualization


table_name = 'top_user_pin'
Top_User_pin['State']=Top_User_pin['State'].apply(convert_state_name)
Top_User_pin.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

# Close the initial MySQL connection
conn.close()

# Reopen the connection using SQLAlchemy for reading the table
conn = engine.connect()

# Read the table into a DataFrame
df = pd.read_sql_table(table_name, con=conn)

# Display the DataFrame
print(df)

# Close the connection
conn.close()
                            

In [ ]:
import os
import json
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# MySQL connection details
host = 'localhost'
user = 'root'
password = '1611'
database = 'PhonePePulse_DVandExp'

# Create a connection to MySQL
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")
conn = engine.connect()

table_name_1 = 'map_trans'
table_name_2='map_user'
table_name_3='agg_transaction'
table_name_4='agg_user'
table_name_5='top_trans_state'
table_name_6='top_trans_dis'
table_name_7='top_trans_pin'
table_name_8='top_user_dis'
table_name_9='top_user_pin'



df_1 = pd.read_sql_table(table_name_1, con=conn)
df_2 = pd.read_sql_table(table_name_2, con=conn)
df_3 = pd.read_sql_table(table_name_3, con=conn)
df_4 = pd.read_sql_table(table_name_4, con=conn)
df_5 = pd.read_sql_table(table_name_5, con=conn)
df_6 = pd.read_sql_table(table_name_6, con=conn)
df_7 = pd.read_sql_table(table_name_7, con=conn)
df_8 = pd.read_sql_table(table_name_8, con=conn)
df_9 = pd.read_sql_table(table_name_9, con=conn)

print(table_name_1)
print(df_1)
print(table_name_2)
print(df_2)
print(table_name_3)
print(df_3)
print(table_name_4)
print(df_4)
print(table_name_5)
print(df_5)
print(table_name_6)
print(df_6)
print(table_name_7)
print(df_7)
print(table_name_8)
print(df_8)
print(table_name_9)
print(df_9)

conn.close()
